In [0]:
from tqdm import tqdm

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import natsort
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dense, Flatten, Dropout
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import ImageDataGenerator
from sklearn import model_selection
from keras.models import load_model
from sklearn.preprocessing import LabelBinarizer
from keras.callbacks import ModelCheckpoint
from keras.layers.advanced_activations import LeakyReLU
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler


from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [46]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [47]:
from zipfile import ZipFile
file_name = "gdrive/My Drive/learning/cloth_prediction/train.zip"
with ZipFile(file_name, 'r') as zip:
  zip.extractall()
  print('done')

done


In [48]:
df = pd.read_csv('gdrive/My Drive/learning/cloth_prediction/train.csv')
test_data = pd.read_csv('gdrive/My Drive/learning/cloth_prediction/test.csv')
submission = pd.read_csv('gdrive/My Drive/learning/cloth_prediction/sample_submission.csv')
test_data.head()
df.head()

,id
0,60001
1,60002
2,60003
3,60004
4,60005


,id,label
0,1,9
1,2,0
2,3,0
3,4,3
4,5,0


In [0]:
# labels_240 = lop

In [0]:
TRAIN_DIR = 'train'

labels_240=[]
i = 0
for img in tqdm(os.listdir(TRAIN_DIR)):
#     path = os.path.join(TRAIN_DIR,img)
    number = int(img[:-4])  
    categ_num = int(df[df.id==number].label)
    labels_240.append(categ_num)
#     print(path)
    i = i+1
    if i > 6000:
     break
    

In [0]:

IMG_SIZE=180
data=[]
# labels_240=[]
i = 0
for img in tqdm(os.listdir(TRAIN_DIR)):
    path = os.path.join(TRAIN_DIR,img)
    img_data = cv2.imread(path)
    img_data = cv2.resize(img_data, (IMG_SIZE,IMG_SIZE))
    image=img_to_array(img_data)
#     image = img_data/255.0
    data.append(image)
    i = i+1
#     print(i)
    if i > 6:
     break

In [53]:
len(data)
len(labels_240)
# labels_240.shape
image.shape

6001

6001

(180, 180, 3)

In [54]:
num_classes=len(np.unique(labels_240))
data=np.array(data,dtype="float32")/255.0
labels_240=np.array(labels_240)
lb=LabelBinarizer()
labels_240=lb.fit_transform(labels_240)
labels_240.shape

(6001, 10)

In [0]:
datagen = ImageDataGenerator(zoom_range = 0.1,
                            height_shift_range = 0.1,
                            width_shift_range = 0.1,
                            rotation_range = 10)

In [0]:
model=Sequential()
model.add(Conv2D(32,(3,3),padding="same",activation="linear",input_shape=(180,180,3)))
model.add(LeakyReLU(alpha=0.1))
model.add(BatchNormalization(axis=-1))
model.add(MaxPooling2D(pool_size=(3,3)))
model.add(Dropout(0.25))

model.add(Conv2D(32,(3,3),padding="same",activation="linear"))
model.add(LeakyReLU(alpha=0.1))
model.add(BatchNormalization(axis=-1))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(64,(3,3),activation='relu'))
model.add(Conv2D(64,(3,3),padding="same",activation="linear"))
model.add(LeakyReLU(alpha=0.1))
model.add(Dropout(0.25))

model.add(Conv2D(128,(3,3),padding="same",activation="linear"))
model.add(LeakyReLU(alpha=0.1))
model.add(BatchNormalization(axis=-1))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(128,(3,3),padding="same",activation="relu"))
model.add(BatchNormalization(axis=-1))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(1024,activation="linear"))
model.add(LeakyReLU(alpha=0.1))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(10
                ,activation="softmax"))

model.compile(loss='categorical_crossentropy', optimizer = Adam(lr=1e-4), metrics=["accuracy"])

In [0]:
x_train,x_test,y_train,y_test=model_selection.train_test_split(data,labels_240,test_size=0.25,random_state=47)

In [58]:
  len(x_train)
  len(y_train)
  len(y_test)
  len(x_test)
  print('')
  len(x_train[0])
  x_train[0].shape
  y_train[0].shape

4500

4500

1501

1501

180

(180, 180, 3)

(10,)

In [0]:
# model.summary()

In [0]:
aug=ImageDataGenerator(rotation_range=25,width_shift_range=0.1,height_shift_range=0.1,shear_range=0.2,horizontal_flip=True,fill_mode="nearest")


In [0]:
checkpoint=ModelCheckpoint(filepath='/model_best2.hdf5',
                           save_best_only=True,verbose=1)


In [0]:
train=model.fit_generator(aug.flow(x_train,y_train,batch_size=64),validation_data=(x_test,y_test),
                          steps_per_epoch=len(x_train)/50,epochs=150,verbose=1,callbacks=[checkpoint])